In [ ]:
import urllib 

params = 'DRIVER={ODBC Driver 13 for for SQLServer};' \
         'SERVER=localhost;' \
         'PORT=1433;' \
         'DATABASE=DbAsftPortal;' \
         'UID=sa;' \
         'PWD=aSj#14776$10036%mM&;'

params = urllib.parse.quote_plus(params)

from sqlalchemy import create_engine
db = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

import pandas as pd

sql = '''
SELECT TABLE_NAME 
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_CATALOG='DbAsftPortal'
'''

pd.read_sql_query(sql, db)
